In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import os

In [ ]:
from datacube import Datacube
from datacube.utils.geometry import CRS
from datacube.api.core import output_geobox
from datacube.storage.masking import make_mask
from odc.ui import show_datasets, with_ui_cbk
from datacube.utils.rio import set_default_rio_config, activate_from_config

set_default_rio_config(aws={'region_name': 'us-west-2'},
                       cloud_defaults=True)

dc = Datacube()

activate_from_config()

In [ ]:
dss = dc.find_datasets(product='ls8_usgs_wofs_scene',
                       lon=(100, 180), lat=(0, -90), # Australian only
                      )
len(dss)

In [ ]:
m = show_datasets(dss, 
                  style={'opacity': 0.3, 'fillOpacity': 0},
                  width='400px', 
                  height='400px', 
                  # zoom=7,
                  scroll_wheel_zoom=False)
m

In [ ]:
all_epsgs = set(ds.crs.epsg for ds in dss)
assert len(all_epsgs) == 1
the_epsg, = all_epsgs
crs = CRS('epsg:{}'.format(the_epsg))

gbox = output_geobox(output_crs=crs,
              resolution=(-30, 30), 
              align=(15, 15), 
              datasets=dss)
gbox

In [ ]:
srcs = dc.group_datasets(dss, 'time')
srcs.time.shape, len(dss)

In [ ]:
%%time
xx = dc.load_data(srcs, gbox, dss[0].type.measurements, 
                  progress_cbk=with_ui_cbk())
xx

In [ ]:
%%time
xx_wet = make_mask(xx.water, wet=True).sum(dim='time', dtype='uint16')

In [ ]:
%%time
xx_dry = make_mask(xx.water, dry=True).sum(dim='time', dtype='uint16')

In [ ]:
%%time
xx_clear = xx_wet + xx_dry
xx_freq = xx_wet.astype('float32')/xx_clear

In [ ]:
fig,ax = plt.subplots(2, 2, figsize=(14,14))

for ax, im, t in zip(ax.ravel(),
                     [xx_wet, xx_dry, xx_clear, xx_freq],
                     ['Wet', 'Dry', 'Clear', 'Freq']):
    ax.imshow(im.values[::4, ::4],
              interpolation='bicubic')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_title(t)

---------------------------------------------------------